In [17]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import asyncio
import joblib

In [18]:
!pwd

/Users/yoshihiroashihara/VFMcalcProj


In [4]:
!ls -la

total 2480
drwxr-xr-x@  27 yoshihiroashihara  staff      864 Aug  1 16:13 .
drwxr-x---+ 145 yoshihiroashihara  staff     4640 Aug  1 15:05 ..
-rw-r--r--@   1 yoshihiroashihara  staff     6148 Aug  1 19:43 .DS_Store
drwxr-xr-x   15 yoshihiroashihara  staff      480 Aug  1 18:34 .git
-rw-r--r--    1 yoshihiroashihara  staff     3078 Jul 30 08:56 .gitignore
-rw-r--r--    1 yoshihiroashihara  staff        7 Aug  1 01:13 .python-version
-rw-r--r--@   1 yoshihiroashihara  staff     1486 Jul 27 17:37 BOJ_ExpInflRate_down.py
-rw-r--r--    1 yoshihiroashihara  staff      505 Aug  1 19:35 Final_Inputs.pkl
-rw-r--r--    1 yoshihiroashihara  staff     4924 Aug  1 15:44 Final_Inputs.py
-rw-r--r--@   1 yoshihiroashihara  staff     7021 Jul 27 17:45 Flet_inputDialogTest.py
-rw-r--r--@   1 yoshihiroashihara  staff     5486 Aug  1 15:44 Flet_inputDialogTest_class.py
-rw-r--r--@   1 yoshihiroashihara  staff     6923 Jul 27 14:21 Flet_kinriselect _test.py
-rw-r--r--@   1 yoshihiroashihara  staff      505

In [56]:
results =joblib.load('results.pkl')
res_PSC_LCC = joblib.load('res_PSC_LCC.pkl')
final_inputs = joblib.load('Final_Inputs.pkl')

In [57]:
final_inputs.keys()
proj_years = final_inputs['proj_years']

In [58]:
res_PSC_LCC.keys()

dict_keys(['LCC_net_expense', 'PSC_net_expense_const_kk', 'PSC_net_expense_ijikanri_kk', 'proj_years', 'const_years', 'ijikanri_years', 'discount_rate'])

In [59]:
const_years = res_PSC_LCC['const_years']
discount_rate = res_PSC_LCC['discount_rate']

In [60]:
results.keys()

dict_keys(['df_PV_cf', 'LCC_discount_factor', 'PSC_const_discount_factor', 'PSC_iji_discount_factor', 'PSC', 'LCC', 'VFM', 'VFM_percent'])

In [61]:
PSC_const_discount_factor = [(1/(1-float(discount_rate)/100))** i for i in reversed(range(int(const_years)))]
LCC_discount_factor = [(1/(1+float(discount_rate)))** i for i in range(1, int(proj_years)+1)]
print(const_years)
print(discount_rate)
print(PSC_const_discount_factor)
print(proj_years)
print(LCC_discount_factor)

TypeError: can only concatenate str (not "int") to str

In [53]:
results['LCC_discount_factor']

[0.23530935016902305,
 0.05537049027696791,
 0.013029194085613528,
 0.0030658911935117976,
 0.0007214328644341916,
 0.0001697598985205865,
 3.994609140563851e-05,
 9.399688810453194e-06,
 2.211834665778778e-06,
 5.204653778857226e-07,
 1.224703698557644e-07,
 2.881842314571983e-08,
 6.781244423315266e-09,
 1.5956902185876268e-09,
 3.7548082840692077e-10,
 8.835414973335898e-11,
 2.079055755849326e-11,
 4.89221258874072e-12,
 1.1511833651452928e-12,
 2.70884209577728e-13]

In [54]:
results['PSC_const_discount_factor']

[1.0335887921389886, 1.0]

In [55]:
results['LCC_discount_factor']

[0.23530935016902305,
 0.05537049027696791,
 0.013029194085613528,
 0.0030658911935117976,
 0.0007214328644341916,
 0.0001697598985205865,
 3.994609140563851e-05,
 9.399688810453194e-06,
 2.211834665778778e-06,
 5.204653778857226e-07,
 1.224703698557644e-07,
 2.881842314571983e-08,
 6.781244423315266e-09,
 1.5956902185876268e-09,
 3.7548082840692077e-10,
 8.835414973335898e-11,
 2.079055755849326e-11,
 4.89221258874072e-12,
 1.1511833651452928e-12,
 2.70884209577728e-13]

In [2]:
JGB_rates = pd.read_csv('https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv', encoding='Shift-JIS', header=1)

In [3]:
JGB_rates.head()

,基準日,1年,2年,3年,4年,5年,6年,7年,8年,9年,10年,15年,20年,25年,30年,40年
0,S49.9.24,10.327,9.362,8.83,8.515,8.348,8.290,8.24,8.121,8.127,-,-,-,-,-,-
1,S49.9.25,10.333,9.364,8.831,8.516,8.348,8.290,8.24,8.121,8.127,-,-,-,-,-,-
2,S49.9.26,10.34,9.366,8.832,8.516,8.348,8.290,8.24,8.122,8.128,-,-,-,-,-,-
3,S49.9.27,10.347,9.367,8.833,8.517,8.349,8.290,8.24,8.122,8.128,-,-,-,-,-,-
4,S49.9.28,10.354,9.369,8.834,8.518,8.349,8.291,8.24,8.122,8.129,-,-,-,-,-,-


In [4]:
def dateConv(date):

    wareki = {
            "R":2018,
            "H":1988,
            "S":1925
            }
  
    date_els = date.split('.', 2)
    year_num = int(date_els[0][1:])
    era = date_els[0][0]
 
    year = year_num + wareki[era]

    month =int(date_els[1])
    day = int(date_els[2])

    dt = datetime(year, month, day)

    return dt



In [5]:
JGB_rates["基準日"] = JGB_rates['基準日'].apply(dateConv)
JGB_rates = JGB_rates.set_index('基準日')
JGB_rates.replace('-', np.nan, inplace=True)

In [6]:
JGB_rates.head()

,1年,2年,3年,4年,5年,6年,7年,8年,9年,10年,15年,20年,25年,30年,40年
基準日,,,,,,,,,,,,,,,
1974-09-24,10.327,9.362,8.83,8.515,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-25,10.333,9.364,8.831,8.516,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-26,10.34,9.366,8.832,8.516,8.348,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-27,10.347,9.367,8.833,8.517,8.349,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-28,10.354,9.369,8.834,8.518,8.349,8.291,8.24,8.122,8.129,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
JGB_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12517 entries, 1974-09-24 to 2023-06-30
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1年      11869 non-null  object 
 1   2年      12156 non-null  object 
 2   3年      12441 non-null  object 
 3   4年      12517 non-null  float64
 4   5年      12517 non-null  float64
 5   6年      12517 non-null  float64
 6   7年      12517 non-null  float64
 7   8年      12517 non-null  float64
 8   9年      12517 non-null  float64
 9   10年     9156 non-null   object 
 10  15年     7815 non-null   object 
 11  20年     9043 non-null   object 
 12  25年     4722 non-null   object 
 13  30年     5840 non-null   object 
 14  40年     3828 non-null   object 
dtypes: float64(6), object(9)
memory usage: 1.5+ MB


In [8]:
JGB_rates['5年'].resample('Y').mean()

基準日
1974-12-31    8.377000
1975-12-31    8.483218
1976-12-31    8.735269
1977-12-31    7.500503
1978-12-31    5.929144
1979-12-31    7.424343
1980-12-31    8.638351
1981-12-31    8.210207
1982-12-31    8.126912
1983-12-31    7.693021
1984-12-31    6.959432
1985-12-31    6.359263
1986-12-31    5.153240
1987-12-31    4.267631
1988-12-31    4.180077
1989-12-31    5.005743
1990-12-31    7.051776
1991-12-31    6.474069
1992-12-31    4.856466
1993-12-31    3.606285
1994-12-31    3.673360
1995-12-31    2.608570
1996-12-31    2.208781
1997-12-31    1.546653
1998-12-31    1.018368
1999-12-31    0.983000
2000-12-31    1.101435
2001-12-31    0.540703
2002-12-31    0.441244
2003-12-31    0.409922
2004-12-31    0.666313
2005-12-31    0.628220
2006-12-31    1.229774
2007-12-31    1.236090
2008-12-31    1.016457
2009-12-31    0.702761
2010-12-31    0.419980
2011-12-31    0.427437
2012-12-31    0.246508
2013-12-31    0.235371
2014-12-31    0.162418
2015-12-31    0.078107
2016-12-31   -0.187139
2017-12

In [9]:
JGB_rate_5_30 = JGB_rates[['5年', '10年', '15年', '20年', '25年', '30年']].dropna()
JGB_rate_5_30 = JGB_rate_5_30.astype(float, errors='raise')

In [10]:
JGB_rate_5_30_12M = JGB_rate_5_30.resample('M').mean().tail(12)
JGB_rate_5_30_12M

,5年,10年,15年,20年,25年,30年
基準日,,,,,,
2022-07-31,0.020400,0.231300,0.597400,0.890300,1.097400,1.238550
2022-08-31,0.001136,0.206818,0.551409,0.824909,1.015545,1.162591
2022-09-30,0.064750,0.269250,0.670050,0.951600,1.160450,1.311800
2022-10-31,0.090850,0.257150,0.769350,1.081450,1.298250,1.456100
2022-11-30,0.087100,0.265300,0.786750,1.104550,1.314900,1.480100
2022-12-31,0.178273,0.357045,0.873182,1.164773,1.351409,1.485909
2023-01-31,0.235947,0.479053,1.054789,1.327947,1.489789,1.575895
2023-02-28,0.214895,0.520053,1.047158,1.310474,1.444000,1.508579
2023-03-31,0.150727,0.402455,0.860500,1.135818,1.265727,1.353545


In [11]:
JGB_rate_5_30_12M.mean()

5年     0.117466
10年    0.358119
15年    0.798023
20年    1.077380
25年    1.249725
30年    1.368891
dtype: float64